# Privacy - target model

reference : https://keras.io/examples/mnist_cnn/

## Imports

In [ ]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

In [ ]:
from __future__ import print_function

%load_ext autoreload
%autoreload 2

import tensorflow as tf

from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model

import numpy as np
import os
import datetime
import matplotlib.pyplot as plt
os.sys.path.append("./src")
from utils import plot_img
from utils import load_mnist_data
from utils import pick_and_show_image

## Data and settings

In [ ]:
batch_size = 32
epochs = 10

img_rows, img_cols = 28, 28
num_classes = 5

In [ ]:
x_train, y_train, x_test, y_test = load_mnist_data("private")

### Plot some sample to check if we have only digits from 0 to 4

In [ ]:
plot_img(x_train, y_train)

## Train model

In [ ]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [ ]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=input_shape))
model.add(Dense(256, activation='relu', input_shape=input_shape))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

In [ ]:
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.SGD(momentum=0.9),
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
ts = datetime.datetime.now().timestamp()

if not "model" in os.listdir("./"):
    os.mkdir("model")
model.save("model/target_model_{}.h5".format(ts))

# Evaluate the model

In [ ]:
model_path = "model/target_model_perceptron.h5"
target_model = load_model(model_path)

### Make a punctual example

In [ ]:
img, label = pick_and_show_image(x_train, y_train)
print("prediction : {}".format(np.argmax(target_model.predict(np.expand_dims(np.expand_dims(img,0),-1)))))

### Evaluate on the whole test set

In [ ]:
predictions_test = target_model.predict(x_test).argmax(axis=-1)
truth_test = y_test.argmax(axis=-1)
np.mean(predictions_test == truth_test)